In [ ]:
## Importing relevant libraries 
import numpy as np
import matplotlib.pyplot as plt 
from nm_lib import nm_lib as nm

# Animation
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

plt.style.use('dark_background')
# plt.style.use('default')

def deriv_cent(xx, hh, **kwargs):
    r"""
    returns the centered 2nd derivative of hh respect to xx. 

    Parameters
    ---------- 
    xx : `array`
        Spatial axis. 
    hh : `array`
        Function that depends on xx. 

    Returns
    -------
    `array`
        The centered 2nd order derivative of hh respect to xx. First 
        and last grid points are ill calculated. 
    """
    
    if kwargs['axis']: 
        return (np.roll(hh, -1, axis=kwargs['axis']) - np.roll(hh, 1, axis=kwargs['axis'])) \
            / (np.roll(xx, -1, axis=kwargs['axis']) - np.roll(xx, 1, axis=kwargs['axis']))
    else:
        return (np.roll(hh, -1) - np.roll(hh, 1)) / (np.roll(xx, -1) - np.roll(xx, 1))

# Hydro-dynamical PDE models

When modeling stellar atmospheres, we are primarely interested in solving the equations of mass, momentum, and energy balance, along with the equations governing the evolution of the magnetic field, the transport equations for the radiation field, heat flux equations, etc. 

For this project, we start with the hydrodynamic equations in 1 dimension as follows: 

$$ \frac{\partial \rho}{\partial t} + \nabla \cdot (\rho \bf u) = 0 $$

$$ \frac{\partial \rho {\bf u}}{\partial t} + \nabla \cdot (\rho {\bf u} \otimes {\bf u}) = - \nabla (P_g)$$

$$ \frac{\partial e}{\partial t } = -\nabla\cdot e {\bf u} -P_g \nabla \cdot {\bf u}$$

where $\rho$, $\bf u$, $P_g$, and $e$ are the density, velocity vector, gas pressure, and internal energy. $\cdot$ and $\otimes$ are the dyadic and tensorial product, respectively. 

In order to connect the pressure with the energy, we use $P_g  = (\gamma-1)e/\rho$ where $\gamma=5/3$. 
<!-- Extra energy-pressure relation -->
The sound speed is $u = p/\rho$, where $p$ is the momentum. 

To start, we find the RHS of the hydrodynamic equations in 1 dimension: 

<span style="color:pink">

In 1D, the RHS of the continuity equation can be written as 

$$
\frac{\partial\rho}{\partial t} + \frac{\partial\rho u}{\partial x} \rightarrow \frac{\partial\rho}{\partial t} = -\frac{\partial\rho u}{\partial x}
$$

Similarely for the momentum equation, 

$$
\frac{\partial\rho u}{\partial t} = -\frac{\partial\rho u^2}{\partial x} - \frac{\partial P g}{\partial x}, 
$$

and for the energy equation we have 

$$
\frac{\partial e}{\partial t} = -\frac{\partial eu}{\partial x} - P_g\frac{\partial u}{\partial x}. 
$$

This means that the fluxes can be written as $F_\rho = \rho u$, $F_v = \rho u^2 + P_g$, and $F_e = u(e + P_g)$. 

</span>

We begin by writing a code for the hydro-dynamical PDE models. 

The RHS of the continuity, momentum and energy equations are implemented in implemented in `step_density`, `step_momentum` and `step_energy`, respectively. 

In [ ]:
def step_density(xx, rho, u, axis=0, cfl_cut=0.98, ddx=lambda x, y: nm.deriv_cent(x, y), bnd_limits=[1,1]): 
    """ 
    Right hand side of the 1D continuity equation, where rho can be a constant or a function of xx. 
    """
    F_rho = rho*u
    dt = cfl_cut*np.min(np.gradient(xx) / np.abs(F_rho))

    return dt, -ddx(xx, F_rho, axis=axis)

def step_momentum(xx, rho, u, Pg, axis=0, cfl_cut=0.98, ddx=lambda x, y: nm.deriv_cent(x, y), bnd_limits=[1,1]): 
    """ 
    Right hand side of the 1D momentm equation, where Pg can be a constant or a function of xx. 
    """
    F_p = rho*u**2 + Pg
    dt = cfl_cut*np.min(np.gradient(xx) / np.abs(F_p))

    return dt, -ddx(xx, F_p, axis=axis)

def step_energy(xx, e, u, Pg, axis=0, cfl_cut=0.98, ddx=lambda x, y: nm.deriv_cent(x, y), bnd_limits=[1,1]
                , source_term=None):
    """ 
    Right hand side of the 1D energy equation, where rho can be a constant or a function of xx. 
    """
    if source_term: 
        F_e = u*(e + Pg) + source_term
    else: 
        F_e = u*(e + Pg)

    dt = cfl_cut*np.min(np.gradient(xx) / np.abs(F_e))

    return dt, -ddx(xx, F_e, axis=axis)

## Building the Code

I have built a hydro-dynamical (HD) numerical code which solves the above set of equations in one dimension. 

<span style="color:red">

TODO: Implement the [Bifrost 6th order spatial derivative, 5th order spatial interpolation](https://github.com/AST-Course/AST5110/wiki/Discretization) with the hyper-diffusion scheme (see [wiki](https://github.com/AST-Course/AST5110/wiki/Hyper-diffusive)). Note that Bifrost is using a [staggered mesh](https://github.com/AST-Course/AST5110/wiki/Staggered-mesh) where the density, pressure, energy, and temperature are cell-centered, and velocity and momentum are at the edges (in 1 dimension). 

</span>

What would you choose for the CFL condition? 

<span style="color:red">

Insert info on CFL condition here. 

</span>

Additionaly, I have made use of Bifrost in order to compare my own code. 

<span style="color:pink">

The time step is important when solving these computational Hydro-Dynamical PDE models. We need to find a balance between computational performance and realistic dynamics, and this is often dependend on the time-step. 

All simulations make assumptions that are impacted by time-step and other factors, such as the solver and mesh cell density. 

The time-step depends on a variety of factors, from mesh cell size, velocity magnitude, solvers and iterations used. 
Additionally, the time-step may be limited due to the computational resources available with a lack of computational processing unit (CPU) power or lack of memory (RAM) for mesh modelling. 

Typically, a large time-step can lead to unstable assumptions and solutions at each step of the solution. The assumptions do not model the flow dynamics correctly and may lead to exponential maximum velocity before the solution fails due to pressure and velocity overload. 

One method for calculating the maximum time-step is the Courant-Friedrichs-Lewy (CFL) condition, 

$$
C = a\frac{\Delta t}{\Delta x}
$$

The CFL condition uses a Courant number, C, calculated from the maximum velocity, a, time-step, $\Delta t$, and the minimum distance between mesh cells, $\Delta x$. The Courant number is a dimensionless number which is a measure of a solution being solved within a mesh cell. Ideally, solutions are solved for each cell avoiding assumptions over multiple mesh cells which can lead to instabilities. The CFL condition is re-arranged to calculate the maximum time-step. A Courant number of 1 will be the maximum theoretical time-step for simulating a stable solution as it is the smallest distance between mesh cells. A Courant number of 0.5 is recommended for an initial guess for a stable solution where it is smaller than the smallest mesh cell-size distance. This should ensure velocity vectors do not exceed several mesh cells and lead to large assumptions and an unstable simulation. 


In this case, the CFL condition is dependent on the 

CFL condition 

sound speed (gamma, pg rho)

We no longer just have `a`, but rather u (osmething plus and minus?)
u+ and - c 

For the timestep, we need to compare all three dt's to get the lowest timestep (`np.min([dt1, dt2, dt3])`). 

I will write one solved using Euler, and another using Lax (with an average of 3). #np roll -1, regular, +1 

xxx talk about the order. 

For Euler, 

$$
f^t(x) = \frac{\partial F(t,x)}{\partial t} = \frac{F(x, t+\Delta t) - F(x, t)}{\Delta t} + \Order (\Delta t^2)
$$

while for Lax, 

xxx 

</span>

Below we evolve the equations in time in `evolv_hydro`. Note that the boundary conditions are now set to `edge`, as wrapping around makes no sense in this case. 

In [ ]:
def evol_hydro(xx, rho, u, Pg, nt, gamma=1.4, axis=0, cfl_cut=0.98, method='euler',
                    ddx=lambda x,y: nm.deriv_cent(x, y), bnd_type='edge', bnd_limits=[1,1], **kwargs):
    """ 
    Evolution of the hydrodynamic equations in time using the Euler method. 

    Parameters
    ----------
    xx : `array`
        Spatial axis.
    rho : `array`
        Density.
    u : `array`
        Velocity.
    e : `array`
        Energy.
    Pg : `array`
        Gas pressure.
    nt : `int`
        Number of time steps.
    axis : `int`
        Axis to be used in the derivative.
    cfl_cut : `float`
        Constant value to limit dt from cfl_adv_burger. 
        By default 0.98
    ddx : `lambda function` 
        Allows to change the space derivative function. 
        By default lambda x,y: deriv_cent(x, y)
    bnd_type : `string` 
        Allows to select the type of boundaries
        by default 'wrap'
    bnd_limits : `list(int)`
        List of two integer elements. The number of pixels that
        will need to be updated with the boundary information. 
        By defalt [1,0]
    
    Returns
    -------
    """
    
    ## Declare the arrays ##
    t       = np.zeros((nt))
    rho_arr = np.zeros((len(xx), nt))
    mom_arr = np.zeros((len(xx), nt))
    e_arr   = np.zeros((len(xx), nt))
    
    ## Initialize variables ##
    rho_arr[:,0] = rho
    mom_arr[:,0] = rho*u
    e_arr[:,0]   = 0.5*rho_arr[:,0]*u**2 + Pg/(gamma-1)
    # e_arr[:,0]   = Pg * rho_arr[:,0] / (gamma - 1) # XXX

    for i in range(nt-1): 

        ## Update variables ## 
        u = mom_arr[:, i] / rho_arr[:, i]
        Pg = (gamma-1) * (e_arr[:, i] - 0.5*rho_arr[:, i] * u**2)
        # Pg = (gamma-1)*e_arr[:,i]/rho_arr[:,i] #XXX

        cs = np.sqrt(gamma*Pg / rho_arr[:,i]) # Sound speed

        ## RHS of the HD equations ##
        dt1, rhs_cont = step_density( xx, rho_arr[:,i], u,        axis=axis, cfl_cut=cfl_cut, ddx=ddx, bnd_limits=bnd_limits)
        dt2, rhs_mom  = step_momentum(xx, mom_arr[:,i], u, Pg=Pg, axis=axis, cfl_cut=cfl_cut, ddx=ddx, bnd_limits=bnd_limits)
        dt3, rhs_e    = step_energy(  xx,   e_arr[:,i], u, Pg=Pg, axis=axis, cfl_cut=cfl_cut, ddx=ddx, bnd_limits=bnd_limits)

        ## CFL condition ##
        dt_sound = np.min(np.abs(np.gradient(xx) / cs))
        dt = cfl_cut*np.min([dt1, dt2, dt3, dt_sound])

        ## Time evolution for the HD equations ##
        if method == 'euler':
            rho_ = rho_arr[:,i]
            mom_ = mom_arr[:,i]
            e_   = e_arr[:,i] 
        elif method == 'lax_2': 
            rho_ = (np.roll(rho_arr[:,i], -1) + np.roll(rho_arr[:,i], 1)) * 0.5
            mom_ = (np.roll(mom_arr[:,i], -1) + np.roll(mom_arr[:,i], 1)) * 0.5
            e_   = (np.roll(  e_arr[:,i], -1) + np.roll(  e_arr[:,i], 1)) * 0.5
        elif method == 'lax_3':
            rho_ = (np.roll(rho_arr[:,i], -1) + rho_arr[:,i] + np.roll(rho_arr[:,i], 1)) / 3
            mom_ = (np.roll(mom_arr[:,i], -1) + mom_arr[:,i] + np.roll(mom_arr[:,i], 1)) / 3
            e_   = (np.roll(  e_arr[:,i], -1) + e_arr[:,i]   + np.roll(  e_arr[:,i], 1)) / 3

        rho_tmp = rho_ + rhs_cont*dt # Density
        mom_tmp = mom_ + rhs_mom*dt  # Momentum
        e_tmp   = e_   + rhs_e*dt    # Energy

        # Boundaries
        if bnd_limits[1] > 0:  # downwind or centered scheme
            rhoBC = rho_tmp[bnd_limits[0]:-bnd_limits[1]]
            momBC = mom_tmp[bnd_limits[0]:-bnd_limits[1]]
            eBC = e_tmp[bnd_limits[0]:-bnd_limits[1]]
        else:
            rhoBC = rho_tmp[bnd_limits[0]:]  # upwind
            momBC = mom_tmp[bnd_limits[0]:]
            eBC = e_tmp[bnd_limits[0]:]

        ## Update arrays witn boundary conditions ##
        rho_arr[:, i+1] = np.pad(rhoBC, bnd_limits, mode=bnd_type)
        mom_arr[:, i+1] = np.pad(momBC, bnd_limits, mode=bnd_type)
        e_arr[:, i+1] = np.pad(eBC, bnd_limits, mode=bnd_type)

        # Updates in time 
        t[i+1] = t[i] + dt
    
    return t, rho_arr, mom_arr, e_arr

## Testing the code 

We will test the code using the Sod-shock tube test [Sod et al. 1978](https://ui.adsabs.harvard.edu/abs/1978JCoPh..27....1S/abstract), a standard test in computational HD codes. It consists of a one-dimensional flow discontinuity problem that provides a good test of a compressible code’s ability to capture shocks and contact discontinuities within a few grid zones and produce the correct density profile in a rarefaction or expansion wave. The test can also be used to check if the code can satisfy the Rankine-Hugoniot shock jump conditions since this test has an analytical solution. If you have access, you can also look at _Computational Gasdynamics book from Culbert B. Laney_ Section 5. However, many other books will describe this problem in detail. 

To test the code, we first compare the analytical solutions using the initial conditions from the wikipedia cite on the Sod Shock Tube, (https://en.wikipedia.org/wiki/Sod_shock_tube). 



We begin by testing this code while Kappa is 0, meaning that the thermal coefficient is not taken into account. 

In [ ]:
def boarders(x, rho_L, rho_R, Pg_L, Pg_R): 
    idx = int(len(x)/2) # Index for setting the boarders 

    # Initialize the arrays 
    rho0 = np.zeros((len(x))) # Density 
    Pg0 = np.zeros((len(x)))  # Pressure 
    u = np.zeros((len(x)))    # Velocity , Note that uL = 0.0, uR = 0.0

    rho0[:idx] = rho_L
    rho0[idx:] = rho_R 
    Pg0[:idx] = Pg_L
    Pg0[idx:] = Pg_R

    return rho0, u, Pg0

In [ ]:
## Define geometric shape 
x = np.linspace(0, 1, 1024)

gamma = 1.4 # 5/3
Pg_L = 1
Pg_R = 0.1
rho_L = 1
rho_R = 0.125

rho0, u, Pg0 = boarders(x, rho_L, rho_R, Pg_L, Pg_R)

plt.close()
plt.plot(x, rho0[:], label=r"Initial $\rho$")
plt.plot(x, Pg0[:], label=r"Initial $Pg$")
plt.xlabel("x")
plt.ylabel("P/rho (normalized)")
plt.legend(); plt.show(); plt.close()

<span style="color:red">

What do we see? 

</span>


## Analytical solution

Below we plot the analytical solution of the density at $t=0.22$

In [ ]:
init = (Pg_L, Pg_R, rho_L, rho_R)

t_end = 0.22
x = np.linspace(0, 1, 1024)
t = np.linspace(0, t_end, 1000) # For animation

rho_a, u_a, e_a = nm.sod_shock_tube_analytical(x, t_end, gamma, init)

fig, ax = plt.subplots(1, 1, figsize=(9, 5))
ax.plot(x, rho_a, label="rho (analytical)")
plt.legend()
plt.show(); plt.close()

Shock -> Discontinuity -> Expansion 

In [ ]:
a_rho = np.zeros((len(x), len(t)))
a_mom = np.zeros((len(x), len(t)))
a_e = np.zeros((len(x), len(t)))

for i in range(len(t)): 
    a_rho[:,i], a_mom[:,i], a_e[:,i] = nm.sod_shock_tube_analytical(x, t[i], gamma, init)
ax.plot(x, a_rho[:,i], label="analytical")

TODO: Plot the velocity (should be 0 throughout)


Note that by using the equation of state, $e = P_g/(\gamma - 1)\rho$, it is possible to compute the intial energy. 


In [ ]:
t_l, rho_new_l, mom_new_l, e_new_l = evol_hydro(x, rho0, u, Pg0, nt=2000, gamma=gamma, axis=0, cfl_cut=0.48, 
                                        method='lax_3', ddx=nm.deriv_cent, bnd_type='edge', bnd_limits=[1,1])

plt.ioff()

M = 10

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))


def init(): 
    axes.plot(x, a_rho[:,0], label="analytical")
    axes.plot(x, rho_new_l[:,0])


def animate(i):
 
    axes.clear()
    axes.plot(x, rho_new_l[:,::M][:,i], label="Lax")
    axes.plot(x, a_rho[:,M*i], label="analytical")
    axes.set_title('Density, t=%.12f'%t[::M][i])
    axes.legend()
    axes.set_xlabel("x")
    axes.set_ylabel("u_i")
    axes.grid()
    
    
anim = FuncAnimation(fig, animate, interval=50, frames=len(t[::M]), init_func=init)
plt.close()
HTML(anim.to_jshtml())

### New initial conditions

Now that the code has been tested with the analytical solution, we attempt another solution. 

The code is set to run a 1D problem using the following initial conditions. The fluid is initially at rest on either side of a density and pressure jump. To the left, respectively right side of the interface, we have: 

$\rho_L = 0.125$

$\rho_R = 1.0$

$Pg_L = 0.125/\gamma$

$Pg_R = 1.0/\gamma$

The ratio of specific heats is chosen to be $\gamma = 5/3$ on both sides of the interface. The units are normalized, with the density and pressure in units of the density and pressure on the left-hand side of the jump and the velocity in units of the sound speed. The length unit is the size of the domain and the time in units of the time required to cross the domain at the speed of sound.

In [ ]:
## Define geometric shape 
x = np.linspace(0, 1, 1024*2) # Size of the domain 

gamma = 5/3 # 5/3
rho_L = 0.125
rho_R = 1.0
Pg_L  = 0.125/gamma
Pg_R  = 1.0/gamma

rho0, u, Pg0 = boarders(x, rho_L, rho_R, Pg_L, Pg_R)
e0 = 0.5*rho0*u**2 + Pg0/(gamma-1)

# plt.plot(x, rho0[:], label=r"Initial $\rho$")
# plt.plot(x, Pg0[:], label=r"Initial $Pg$")
# # plt.plot(x, e0[:], label=r"Initial $e$")
# plt.xlabel("x")
# plt.ylabel("P/rho (normalized)")
# plt.legend(); plt.show(); plt.close()


t_l, rho_new_l, mom_new_l, e_new_l = evol_hydro(x, rho0, u, Pg0, nt=2000, gamma=gamma, axis=0, cfl_cut=0.48, 
                                        method='lax_3', ddx=nm.deriv_cent, bnd_type='edge', bnd_limits=[1,1])

                             
plt.ioff()

M = 20

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

def init(): 
    # normalize
    # axes.plot(x, rho_new_e[:,0])
    axes.plot(x, rho_new_l[:,0])
    # axes.plot(x, mom_new[:,0])
    # axes.plot(x, e_new[:,0])

def animate(i):
    axes.clear()
    # axes.plot(x, rho_new_e[:,::M][:,i], label="Euler")
    axes.plot(x, rho_new_l[:,::M][:,i], label="Lax")
    # axes.plot(x, mom_new[:,::M][:,i], label="momentum")
    # axes.plot(x, e_new[:,::M][:,i], label="energy")
    axes.set_title('t=%.12f'%t_l[::M][i])
    rho0 = np.full((len(x), 1), rho_L)
    Pg0 = np.full((len(x), 1), Pg_L)
    axes.set_xlabel("x")
    axes.set_ylabel("Density")
    axes.set_ylim(0, 1)
    axes.legend()
    axes.grid()
    
    
anim = FuncAnimation(fig, animate, interval=50, frames=len(t_l[::M]), init_func=init)
plt.close()
HTML(anim.to_jshtml())